In [4]:
import keras 
import numpy as np 
import pandas as pd 
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature 

In [5]:
# Loading the dataset 
data=  pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep= ";")

data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


<h2>Train Test Split</h2>

In [17]:
train, test  = train_test_split(data, test_size= 0.25, random_state= 42)

In [21]:
#  Train Data
train_x = train.drop(['quality'],axis=1).values
train_y = train[['quality']].values.ravel()     # here .ravel is use to convert the 2d array into 1d array

#  Test Data
test_x = test.drop(['quality'],axis=1).values 
test_y = test[['quality']].values.ravel()       # here .ravel is use to convert the 2d array into 1d array

#  splitting the Train data further into Training and Validation data
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size= 0.2, random_state=42)

signature = infer_signature(train_x, train_y)

In [46]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [47]:
def objective(params):
    # MLFLOW will track the parameters and results for each run 
    result = train_model(
        params, 
        epochs= 3,
        train_x = train_x,
        train_y = train_y,
        valid_x = valid_x,
        valid_y = valid_y,
        test_x = test_x,
        test_y = test_y,
    )
    return result 

In [48]:
space= {
    'lr': hp.loguniform('lr', np.log(1e-5), np.log(1e-1)),
    'momentum': hp.uniform("moemntum", 0.0, 1.0)   
}

In [49]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                                                                             

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1:17 2s/step - loss: 37.3060 - root_mean_squared_error: 6.1079
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 37.1267 - root_mean_squared_error: 6.0930 
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 36.6673 - root_mean_squared_error: 6.0553
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 36.5549 - root_mean_squared_error: 6.0460
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 36.3628 - root_mean_squared_error: 6.0302 - val_loss: 35.6406 - val_root_mean_squared_error: 5.9700

Epoch 2/3                                                                                             

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 35.4727 - root_mean_squared_error: 5.9559
19/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 35.6886 - root_mean_squared_error: 5.9740 
37/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 35.4430 - root_mean_squared_error: 5.9533
46/46 ━━━━━━━━━━

2025/08/09 13:47:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                                             

 1/46 ━━━━━━━━━━━━━━━━━━━━ 44s 996ms/step - loss: 36.4512 - root_mean_squared_error: 6.0375
11/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 20.8378 - root_mean_squared_error: 4.4943   
23/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 15.1680 - root_mean_squared_error: 3.7939
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 12.5044 - root_mean_squared_error: 3.4177
44/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.8795 - root_mean_squared_error: 3.1666
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 4.7042 - root_mean_squared_error: 2.1689 - val_loss: 0.6779 - val_root_mean_squared_error: 0.8234

Epoch 2/3                                                                                             

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 0.7323 - root_mean_squared_error: 0.8557
12/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6736 - root_mean_squared_error: 0.8205 
26/46 ━━━━━━━━━━

2025/08/09 13:48:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                                             

 1/46 ━━━━━━━━━━━━━━━━━━━━ 26s 592ms/step - loss: 41.4300 - root_mean_squared_error: 6.4366
18/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.4356 - root_mean_squared_error: 6.0339   
35/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 33.8596 - root_mean_squared_error: 5.8127
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 26.9859 - root_mean_squared_error: 5.1948 - val_loss: 17.8687 - val_root_mean_squared_error: 4.2271

Epoch 2/3                                                                                             

 1/46 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - loss: 17.2276 - root_mean_squared_error: 4.1506
 8/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 16.5273 - root_mean_squared_error: 4.0651 
19/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 16.1591 - root_mean_squared_error: 4.0194
20/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 16.1034 - root_mean_squared_error: 4.0123
23/46 ━━━━━━

...

Epoch 1/3                                                                                             

 1/46 ━━━━━━━━━━━━━━━━━━━━ 24s 547ms/step - loss: 34.4465 - root_mean_squared_error: 5.8691
23/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31.0288 - root_mean_squared_error: 5.5687   
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 24.6516 - root_mean_squared_error: 4.9650 - val_loss: 17.7495 - val_root_mean_squared_error: 4.2130

Epoch 2/3                                                                                             

 1/46 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 16.1240 - root_mean_squared_error: 4.0155
13/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 16.8970 - root_mean_squared_error: 4.1104 
28/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 16.1013 - root_mean_squared_error: 4.0111
41/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 15.4289 - root_mean_squared_error: 3.9248
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 13.2254 - root_mean_squared_error: 3.6367 - val_loss: 

2025/08/09 13:49:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|█████████████████████████████████| 4/4 [02:07<00:00, 31.99s/trial, best loss: 0.7408732771873474]

2025/08/09 13:49:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.01165934844062304), 'moemntum': np.float64(0.9055850064555652)}
Best eval rmse: 0.7408732771873474
